## Ejercicio 1
Considerando los logs de acciones realizadas sobre ítems, mostrar el top 10 de ids.

## Preparando sesión de Spark

### Spark session y Spark context

In [1]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Lectura de archivos

In [2]:
#import pandas as pd
import numpy as np 

In [3]:
#texts = pd.read_csv('./WikipediaDataset/logs.csv', usecols = ["action", "logtype", "title"])
#texts.to_parquet('./WikipediaDataset/logs_ActionLogtypeTitle.parquet')
#del texts
#Dataset de logs.parquet fue creado con las líneas anteriores, y ahora puede ser leído de la siguiente forma:
dataframeLogs = sqlContext.read.parquet('./WikipediaDataset/logs_ActionLogtypeTitle.parquet')
rddLogs = dataframeLogs.rdd

In [4]:
#texts = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["title", "id"])
#texts.to_parquet('./WikipediaDataset/contentsTitleId.parquet')
#del texts
dataframeContents = sqlContext.read.parquet('./WikipediaDataset/contentsTitleId.parquet')
rddContents = dataframeContents.rdd

## Preparando y limpiando los datasets

Archivo Logs

In [5]:
rddLogs

MapPartitionsRDD[7] at javaToPython at NativeMethodAccessorImpl.java:0

In [6]:
rddLogs1 = rddLogs

In [7]:
rddLogs1.take(5)

[Row(logtype='delete', action='delete', title='Verificacionismo'),
 Row(logtype='delete', action='delete', title='Rapunzel'),
 Row(logtype='delete', action='delete', title='Macromedia Flash'),
 Row(logtype='delete', action='delete', title='Archivo:Gnu head.png'),
 Row(logtype='delete', action='delete', title='Archivo:Gnu-head-sm.jpg')]

In [8]:
rddLogs1.count()

12188946

In [9]:
rddLogs1 = rddLogs1.filter(lambda x: ((x[0] != None) and (x[1] != None) and (x[2] != None)))

In [10]:
rddLogs1.count()

12186939

In [11]:
rddTitulos = rddLogs1.map(lambda x: (x[2], 1))
rddTitulos.take(5)

[('Verificacionismo', 1),
 ('Rapunzel', 1),
 ('Macromedia Flash', 1),
 ('Archivo:Gnu head.png', 1),
 ('Archivo:Gnu-head-sm.jpg', 1)]

In [12]:
rddTitulos = rddTitulos.reduceByKey(lambda x, y: x + y)
rddTitulos.take(5)

[('Sonchus acaulis', 2),
 ('Archivo:11M1.JPG', 2),
 ('Yubaba', 2),
 ('Mudhoney', 1),
 ('Cacabelos (León)', 1)]

Ya el rddTitulos está listo para hacer un join.

Se procede ahora con el rdd del archivo contents.

In [13]:
rddContents

MapPartitionsRDD[15] at javaToPython at NativeMethodAccessorImpl.java:0

In [20]:
rddContents1 = rddContents

In [15]:
rddContents1.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5),
 Row(title='Andorra', id=7),
 Row(title='Argentina', id=10),
 Row(title='Geografía de Andorra', id=15),
 Row(title='Demografía de Andorra', id=17)]

In [16]:
rddContents1.count()

4132164

In [17]:
rddContents1 = rddContents1.filter(lambda x: ((x[0] != None) and (x[1] != None)))
rddContents1.count()

4132160

In [18]:
rddContents1 = rddContents.map(lambda x: (x[0], x[1]))
rddContents1.take(5)

[('Wikipedia:Artículos solicitados', 5),
 ('Andorra', 7),
 ('Argentina', 10),
 ('Geografía de Andorra', 15),
 ('Demografía de Andorra', 17)]

## Desarrollo

Se procede a hacer un inner join para quedarse con la intersección de ambos rdd. Como tanto en el rdd de losgs y el rdd de contents no hay títulos repetidos, esta operación no dará filas con títulos repetidos. 

In [21]:
rddJoin = rddTitulos.join(rddContents1)
rddJoin.count()

1193007

In [22]:
rddJoin.take(5)

[('Librería Nacional de Medicina de los Estados Unidos', (2, 2344816)),
 ('Copa Nacional de San Cristóbal y Nieves', (2, 9782925)),
 ('Claudio Zolla', (2, 9795812)),
 ('Aldo Bufi Landi', (2, 9850508)),
 ('Venom', (4, 224692))]

Finalmente se pueden ordenar los contenidos según la cantidad de veces de que el título aparece en logs. Se procede entonces a quedarse con los 10 primeros que aparecen más en logs.

In [23]:
listTop10 = rddJoin.takeOrdered(10, lambda x: -x[1][0]) 
print(listTop10)

[('Wikipedia:Portada', (521, 2271189)), ('Wikipedia:Vandalismo en curso', (343, 19013)), ('Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Miscelánea/Actual', (324, 951915)), ('Plantilla:Ficha de entidad subnacional', (305, 314958)), ('Wikipedia:Informes de error', (159, 4074354)), ('Anexo:Wikipedias', (159, 208803)), ('Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Violaciones de etiqueta/Actual', (147, 1725457)), ('Anexo:Pokémon de la primera generación', (130, 810532)), ('Bandera del Reino Unido', (113, 41189)), ('Wikipedia:Café/Archivo/Miscelánea/Actual', (104, 514941))]


## Conclusión
Se puede observar que los contenidos con más cantidad de logs están relacionados a los contenidos pertenecientes a las subcategorías de 'Wikipedia:', 'Anexo:'.  

## En limpio
A lo largo de la consigna se fuerpn descubriendo los rdds para ir verificando cuáles operaciones hacer y que todo funcione correctamente. Con esto en mente está claro que la consigna puede ser resuelta con menos líneas de código, por lo que se procede a mostrar una solución 'final', con los rdds recién leídos y cargados, saltándose los pasos intermedios de investigación de los rdds.

Las siguientes transformaciones y acciones están hechas sobre los rdds originales obtenidos al hacer la lectura de los archivos correspondientes.

In [24]:
rddTitulos = rddLogs.filter(lambda x: ((x[0] != None) and (x[1] != None) and (x[2] != None)))\
.map(lambda x: (x[2], 1)).reduceByKey(lambda x, y: x + y)

rddContents = rddContents.filter(lambda x: ((x[0] != None) and (x[1] != None))).map(lambda x: (x[0], x[1]))

In [ ]:
print(rddTitulos.join(rddContents).takeOrdered(10, lambda x: -x[1][0]))